# Classification de documents : prise en main des outils

Le but de ce TP est de classer des documents textuels... Dans un premier temps, nous allons vÃ©rifier le bon fonctionnement des outils sur des donnÃ©es jouets puis appliquer les concepts sur des donnÃ©es rÃ©elles.


## Conception de la chaine de traitement
Pour rappel, une chaine de traitement de documents classique est composÃ©e des Ã©tapes suivantes:
1. Lecture des donnÃ©es et importation
    - Dans le cadre de nos TP, nous faisons l'hypothÃ¨se que le corpus tient en mÃ©moire... Si ce n'est pas le cas, il faut alors ajouter des structures de donnÃ©es avec des buffers (*data-reader*), bien plus complexes Ã  mettre en place.
    - Le plus grand piÃ¨ge concerne l'encodage des donnÃ©es. Dans le TP... Pas (ou peu) de problÃ¨me. Dans la vraie vie: il faut faire attention Ã  toujours maitriser les formats d'entrÃ©e et de sortie.
1. Traitement des donnÃ©es brutes paramÃ©trique. Chaque traitement doit Ãªtre activable ou desactivable + paramÃ©trable si besoin.
    - Enlever les informations *inutiles* : chiffre, ponctuations, majuscules, etc... <BR>
    **L'utilitÃ© dÃ©pend de l'application!**
    - Segmenter en mots (=*Tokenization*)
    - Elimination des stop-words
    - Stemming/lemmatisation (racinisation)
    - Byte-pair encoding pour trouver les mots composÃ©s (e.g. Sorbonne UniversitÃ©, Ville de Paris, Premier Ministre, etc...)
1. Traitement des donnÃ©es numÃ©riques
    - Normalisation *term-frequency* / binarisation
    - Normalisation *inverse document frequency*
    - Elimination des mots rares, des mots trop frÃ©quents
    - Construction de critÃ¨re de sÃ©parabilitÃ© pour Ã©liminer des mots etc...
1. Apprentissage d'un classifieur
    - Choix du type de classifieur
    - RÃ©glage des paramÃ¨tres du classifieur (rÃ©gularisation, etc...)

## Exploitation de la chaine de traitement

On appelle cette Ã©tape la rÃ©alisation d'une campagne d'expÃ©riences: c'est le point clÃ© que nous voulons traviller en TAL cette annÃ©e.
1. Il est impossible de tester toutes les combinaisons par rapport aux propositions ci-dessus... Il faut donc en Ã©liminer un certain nombre.
    - En discutant avec les experts mÃ©tiers
    - En faisant des tests prÃ©liminaires
1. AprÃ¨s ce premier filtrage, il faut:
    - Choisir une Ã©valuation fiable et pas trop lente (validation croisÃ©e, leave-one-out, split apprentissage/test simple)
    - Lancer des expÃ©riences en grand
        - = *grid-search*
        - parallÃ¨liser sur plusieurs machines
        - savoir lancer sur un serveur et se dÃ©connecter
1. Collecter et analyser les rÃ©sultats


## InfÃ©rence

L'infÃ©rence est ensuite trÃ¨s classique: la chaine de traitement optimale est apte Ã  traiter de nouveaux documents

# Etape 1: charger les donnÃ©es

In [18]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

In [10]:
# Chargement des donnÃ©es:
def load_pres(fname):
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour rÃ©gler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
        if lab.count('M') >0:
            alllabs.append(-1)
        else: 
            alllabs.append(1)
        alltxts.append(txt)
    return alltxts,alllabs


In [11]:
fname = "/Users/vguigue/Documents/Cours/TAL/tp/tme2/ressources/AFDpresidentutf8/corpus.tache1.learn.utf8"

alltxts,alllabs = load_pres(fname)


In [12]:
print(len(alltxts),len(alllabs))
print(alltxts[0])
print(alllabs[0])
print(alltxts[-1])
print(alllabs[-1])



57413 57413
 Quand je dis chers amis, il ne s'agit pas lÃ  d'une formule diplomatique, mais de l'expression de ce que je ressens.

1
 Je compte sur vous.

1


In [22]:
def load_movies(path2data): # 1 classe par rÃ©pertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un rÃ©pertoire
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg rÃ©pertoire = cht classe
        
    return alltxts,labs


In [23]:
path = "/Users/vguigue/Documents/Cours/TAL/tp/tme2/ressources/movies/movies1000/"

alltxts,alllabs = load_movies(path)

In [24]:
print(len(alltxts),len(alllabs))
print(alltxts[0])
print(alllabs[0])
print(alltxts[-1])
print(alllabs[-1])

2000 2000
bad . bad . 
bad . 
that one word seems to pretty much sums up beyond the valley of the dolls . 
if that summary isn't enough for you , how about t&a , t&a , t&a ? 
still haven't got the point ? 
other than director russ meyer's predilection for casting attractive large breasted women who ultimately expose the afore-mentioned anatomical areas , there is really only one other reason to recommend even taking a look at this movie . 
that is the fact that it was co-written by famed film critic roger ebert , who also was responsible for the screenplay . 
after watching this movie you will never be able to sit through another one of his reviews where he gives a movie a thumbs down for bad writing with a straight face . 
this movie stinks out loud . 
quite frankly , this movie deserves a . 
but there are parts of it that are so bad they are almost funny . 
so i'm giving it a . 
and maybe that is too generous . 
right from the opening credits , i knew that i had a class-a bomb on my 

# Transformation paramÃ©trique du texte

Vous devez tester, par exemple, les cas suivants:
- transformation en minuscule ou pas
- suppression de la ponctuation
- transformation des mots entiÃ¨rement en majuscule en marqueurs spÃ©cifiques
- suppression des chiffres ou pas
- conservation d'une partie du texte seulement (seulement la premiÃ¨re ligne = titre, seulement la derniÃ¨re ligne = rÃ©sumÃ©, ...)
- stemming
- ...


VÃ©rifier systÃ©matiquement sur un exemple ou deux le bon fonctionnement des mÃ©thodes sur deux documents (au moins un de chaque classe).

# Extraction du vocabulaire

Exploration prÃ©liminaire des jeux de donnÃ©es.

- Quelle est la taille d'origine du vocabulaire?
- Que reste-t-il si on ne garde que les 100 mots les plus frÃ©quents? [word cloud]
- Quels sont les 100 mots dont la frÃ©quence documentaire est la plus grande? [word cloud]
- Quels sont les 100 mots les plus discriminants au sens de odds ratio? [word cloud]
- Quelle est la distribution d'apparition des mots (Zipf)
- Quels sont les 100 bigrammes/trigrammes les plus frÃ©quents?


Question qui devient de plus en plus intÃ©ressante avec les approches modernes:
est-il possible d'extraire des tri-grammes de lettres pour reprÃ©senter nos documents?

Quelle performances attendrent? Quels sont les avantages et les inconvÃ©nients d'une telle approche?

# ModÃ¨les de Machine Learning

Avant de lancer de grandes expÃ©riences, il faut se construire une base de travail solide en Ã©tudiant les questions suivantes:

- Combien de temps Ã§a prend d'apprendre un classifieur NB/SVM/RegLog sur ces donnÃ©es en fonction de la taille du vocabulaire?
- La validation croisÃ©e est-elle nÃ©cessaire? Est ce qu'on obtient les mÃªmes rÃ©sultats avec un simple *split*?
- La validation croisÃ©e est-elle stable? A partir de combien de fold (travailler avec diffÃ©rentes graines alÃ©atoires et faire des statistiques basiques)?

## PremiÃ¨re campagne d'expÃ©riences

Les techniques sur lesquelles nous travaillons Ã©tant sujettes au sur-apprentissage: trouver le paramÃ¨tre de rÃ©gularisation dans la documentation et optimiser ce paramÃ¨tre au sens de la mÃ©trique qui vous semble la plus appropriÃ©e (cf question prÃ©cÃ©dente).

## Equilibrage des donnÃ©es

Un problÃ¨me reconnu comme dur dans la communautÃ© est celui de l'Ã©quilibrage des classes (*balance* en anglais). Que faire si les donnÃ©es sont Ã  80, 90 ou 99% dans une des classes?
Le problÃ¨me est dur mais frÃ©quent; les solutions sont multiples mais on peut isoler 3 grandes familles de solution.

1. RÃ©-Ã©quilibrer le jeu de donnÃ©es: supprimer des donnÃ©es dans la classe majoritaire et/ou sur-Ã©chantilloner la classe minoritaire.<BR>
   $\Rightarrow$ A vous de jouer pour cette technique
1. Changer la formulation de la fonction de coÃ»t pour pÃ©naliser plus les erreurs dans la classe minoritaire:
soit une fonction $\Delta$ mesurant les Ã©carts entre $f(x_i)$ et $y_i$ 
$$C = \sum_i  \alpha_i \Delta(f(x_i),y_i), \qquad \alpha_i = \left\{
\begin{array}{ll}
1 & \mbox{si } y_i \in \mbox{classe majoritaire}\\
B>1 & \mbox{si } y_i \in \mbox{classe minoritaire}\\
\end{array} \right.$$
<BR>
   $\Rightarrow$ Les SVM et d'autres approches sklearn possÃ¨dent des arguments pour rÃ©gler $B$ ou $1/B$... Ces arguments sont utiles mais pas toujours suffisant.
1. Courbe ROC et modification du biais. Une fois la fonction $\hat y = f(x)$ apprise, il est possible de la *bidouiller* a posteriori: si toutes les prÃ©dictions $\hat y$ sont dans une classe, on va introduire $b$ dans $\hat y = f(x) + b$ et le faire varier jusqu'Ã  ce qu'un des points change de classe. On peut ensuite aller de plus en plus loin.
Le calcul de l'ensemble des scores associÃ©s Ã  cette approche mÃ¨ne directement Ã  la courbe ROC.

**Note:** certains classifieurs sont intrinsÃ¨quement plus rÃ©sistante au problÃ¨me d'Ã©quilibrage, c'est par exemple le cas des techniques de gradient boosting que vous verrez l'an prochain.